# 🔬 Ensemble Model Weight Optimization
## หาค่า Weight ที่เหมาะสมสำหรับ 3 โมเดล

**เป้าหมาย:**
- ทดสอบ 3 โมเดล (Xception, F3Net, Effort-CLIP)
- หาค่า weight ที่ให้ความแม่นยำสูงสุด
- ประหยัด Compute Units (ใช้ประมาณ 5-10 units)

**Dataset แนะนำ:**
- FaceForensics++ (test set)
- Celeb-DF (test set)
- หรือ custom dataset ของคุณ

## 📦 Setup & Installation

In [ ]:
# ตรวจสอบ GPU
!nvidia-smi

In [ ]:
# ติดตั้ง dependencies
!pip install torch torchvision timm pillow scikit-learn matplotlib seaborn tqdm
!pip install facenet-pytorch
!pip install git+https://github.com/openai/CLIP.git

In [ ]:
# อัปโหลดโปรเจกต์ไปยัง Colab
from google.colab import drive
drive.mount('/content/drive')

# หรือใช้ git clone
# !git clone <your-repo-url>

# หรืออัปโหลดไฟล์ model weights แบบ manual
from google.colab import files
print("📁 อัปโหลดไฟล์ model weights (3 ไฟล์):")
print("1. xception_best.pth")
print("2. f3net_best.pth")
print("3. effort_clip_L14_trainOn_FaceForensic.pth")

## 🏗️ โหลดโมเดลจากโปรเจกต์

In [ ]:
import sys
import os

# เพิ่ม path ไปยังโปรเจกต์
sys.path.insert(0, '/content/deepfake-detection/backend/app')  # ปรับ path ตามที่คุณวาง

import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import numpy as np
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔧 Using device: {device}")

In [ ]:
# โหลดโมเดลจากโปรเจกต์
from models.xception_model import XceptionModel
from models.f3net_model import F3NetModel
from models.effort_model import EffortModel

# โหลด 3 โมเดล
print("📥 Loading models...")

xception = XceptionModel('models/weights/xception_best.pth', device)
print("✅ Xception loaded")

f3net = F3NetModel('models/weights/f3net_best.pth', device)
print("✅ F3Net loaded")

effort = EffortModel('models/weights/effort_clip_L14_trainOn_FaceForensic.pth', device)
print("✅ Effort-CLIP loaded")

models = {
    'xception': xception,
    'f3net': f3net,
    'effort': effort
}

print(f"\n🎯 Total models loaded: {len(models)}")

## 📊 เตรียม Test Dataset

In [ ]:
# Option 1: ใช้ dataset จาก Drive
# ควรมี folder structure:
# test_data/
#   ├── real/
#   │   ├── img1.jpg
#   │   ├── img2.jpg
#   └── fake/
#       ├── img1.jpg
#       ├── img2.jpg

from pathlib import Path
import glob

# ปรับ path ตาม dataset ของคุณ
TEST_DATA_PATH = '/content/drive/MyDrive/test_data'  # ปรับตามที่คุณเก็บ

# หา path ของรูปทั้งหมด
real_images = glob.glob(f'{TEST_DATA_PATH}/real/*.jpg') + glob.glob(f'{TEST_DATA_PATH}/real/*.png')
fake_images = glob.glob(f'{TEST_DATA_PATH}/fake/*.jpg') + glob.glob(f'{TEST_DATA_PATH}/fake/*.png')

print(f"📊 Dataset Summary:")
print(f"  Real images: {len(real_images)}")
print(f"  Fake images: {len(fake_images)}")
print(f"  Total: {len(real_images) + len(fake_images)}")

# สร้าง dataset list
test_data = []
for img_path in real_images:
    test_data.append({'path': img_path, 'label': 0})  # 0 = REAL
for img_path in fake_images:
    test_data.append({'path': img_path, 'label': 1})  # 1 = FAKE

print(f"\n✅ Test dataset ready: {len(test_data)} images")

In [ ]:
# Preprocessing (ตามที่ใช้ในโปรเจกต์)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Face detector (optional - ถ้าต้องการ crop face)
from facenet_pytorch import MTCNN
face_detector = MTCNN(keep_all=False, device='cpu', post_process=False, min_face_size=40)
print("✅ Preprocessing ready")

## 🔬 ทดสอบโมเดลแต่ละตัว

In [ ]:
def predict_single_model(model, image_tensor):
    """ทำนายด้วยโมเดลเดียว"""
    with torch.no_grad():
        fake_prob, real_prob = model.predict(image_tensor)
    return fake_prob, real_prob

def evaluate_model(model, test_data, model_name):
    """ประเมินโมเดลเดียว"""
    print(f"\n🔍 Evaluating {model_name}...")
    
    predictions = []
    labels = []
    
    for item in tqdm(test_data, desc=f"{model_name}"):
        try:
            # โหลดและ preprocess
            img = Image.open(item['path']).convert('RGB')
            
            # Optional: crop face
            # boxes, _ = face_detector.detect(img)
            # if boxes is not None:
            #     x1, y1, x2, y2 = map(int, boxes[0])
            #     img = img.crop((x1, y1, x2, y2))
            
            img_tensor = transform(img).unsqueeze(0).to(device)
            
            # ทำนาย
            fake_prob, real_prob = predict_single_model(model, img_tensor)
            
            predictions.append(fake_prob)
            labels.append(item['label'])
            
        except Exception as e:
            print(f"⚠️  Error processing {item['path']}: {e}")
            continue
    
    return np.array(predictions), np.array(labels)

In [ ]:
# ทดสอบทั้ง 3 โมเดล
results = {}

for model_name, model in models.items():
    predictions, labels = evaluate_model(model, test_data, model_name)
    results[model_name] = {
        'predictions': predictions,
        'labels': labels
    }
    
    # คำนวณ accuracy
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    
    pred_labels = (predictions > 0.5).astype(int)
    acc = accuracy_score(labels, pred_labels)
    prec = precision_score(labels, pred_labels)
    rec = recall_score(labels, pred_labels)
    f1 = f1_score(labels, pred_labels)
    
    results[model_name]['metrics'] = {
        'accuracy': acc,
        'precision': prec,
        'recall': rec,
        'f1': f1
    }
    
    print(f"\n📊 {model_name} Performance:")
    print(f"  Accuracy:  {acc:.4f}")
    print(f"  Precision: {prec:.4f}")
    print(f"  Recall:    {rec:.4f}")
    print(f"  F1 Score:  {f1:.4f}")

## 🎯 หาค่า Weight ที่เหมาะสม

In [ ]:
from itertools import product

def evaluate_ensemble(weights, results):
    """ประเมิน ensemble ด้วย weight ที่กำหนด"""
    w_xception, w_f3net, w_effort = weights
    
    # รวม predictions จาก 3 โมเดล
    ensemble_pred = (
        results['xception']['predictions'] * w_xception +
        results['f3net']['predictions'] * w_f3net +
        results['effort']['predictions'] * w_effort
    )
    
    labels = results['xception']['labels']
    pred_labels = (ensemble_pred > 0.5).astype(int)
    
    from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
    
    acc = accuracy_score(labels, pred_labels)
    f1 = f1_score(labels, pred_labels)
    auc = roc_auc_score(labels, ensemble_pred)
    
    return {
        'accuracy': acc,
        'f1': f1,
        'auc': auc
    }

# Grid search สำหรับ weights
print("🔍 Searching for optimal weights...\n")

# สร้างชุด weights ที่เป็นไปได้ (รวมต้องเป็น 1.0)
step = 0.05  # ขั้น 0.05 (5%)
weight_range = np.arange(0.0, 1.0 + step, step)

best_score = 0
best_weights = None
best_metrics = None

all_results = []

for w1 in weight_range:
    for w2 in weight_range:
        w3 = 1.0 - w1 - w2
        
        # ตรวจสอบว่า w3 อยู่ในช่วงที่ถูกต้อง
        if w3 < 0 or w3 > 1.0 or abs(w1 + w2 + w3 - 1.0) > 0.01:
            continue
        
        weights = (w1, w2, w3)
        metrics = evaluate_ensemble(weights, results)
        
        # ใช้ F1 score เป็นตัวตัดสิน (หรือเลือก accuracy/auc)
        score = metrics['f1']
        
        all_results.append({
            'weights': weights,
            'metrics': metrics,
            'score': score
        })
        
        if score > best_score:
            best_score = score
            best_weights = weights
            best_metrics = metrics

print("\n" + "="*50)
print("🏆 BEST ENSEMBLE CONFIGURATION")
print("="*50)
print(f"\n📊 Optimal Weights:")
print(f"  Xception:    {best_weights[0]:.3f}")
print(f"  F3Net:       {best_weights[1]:.3f}")
print(f"  Effort-CLIP: {best_weights[2]:.3f}")
print(f"\n📈 Performance:")
print(f"  Accuracy: {best_metrics['accuracy']:.4f}")
print(f"  F1 Score: {best_metrics['f1']:.4f}")
print(f"  AUC:      {best_metrics['auc']:.4f}")
print("="*50)

## 📊 Visualization

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 1. แสดงประสิทธิภาพแต่ละโมเดล
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

model_names = ['Xception', 'F3Net', 'Effort-CLIP']
metrics_names = ['accuracy', 'precision', 'recall', 'f1']

for idx, (model_name, model_key) in enumerate(zip(model_names, ['xception', 'f3net', 'effort'])):
    metrics = results[model_key]['metrics']
    values = [metrics[m] for m in metrics_names]
    
    axes[idx].bar(metrics_names, values, color=['#3498db', '#e74c3c', '#2ecc71', '#f39c12'])
    axes[idx].set_title(f'{model_name} Performance', fontsize=12, fontweight='bold')
    axes[idx].set_ylim([0, 1])
    axes[idx].set_ylabel('Score')
    
    for i, v in enumerate(values):
        axes[idx].text(i, v + 0.02, f'{v:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.savefig('individual_model_performance.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# 2. Weight optimization heatmap (สำหรับ 2 โมเดลแรก)
# สร้าง grid สำหรับ visualization
weight_grid = {}
for result in all_results:
    w1, w2, w3 = result['weights']
    key = (round(w1, 2), round(w2, 2))
    if key not in weight_grid:
        weight_grid[key] = result['metrics']['f1']

# สร้าง heatmap data
w1_vals = sorted(list(set([k[0] for k in weight_grid.keys()])))
w2_vals = sorted(list(set([k[1] for k in weight_grid.keys()])))

heatmap_data = np.zeros((len(w2_vals), len(w1_vals)))
for i, w2 in enumerate(w2_vals):
    for j, w1 in enumerate(w1_vals):
        key = (w1, w2)
        if key in weight_grid:
            heatmap_data[i, j] = weight_grid[key]
        else:
            heatmap_data[i, j] = np.nan

plt.figure(figsize=(12, 8))
sns.heatmap(heatmap_data, xticklabels=[f'{w:.2f}' for w in w1_vals], 
            yticklabels=[f'{w:.2f}' for w in w2_vals],
            cmap='RdYlGn', annot=False, cbar_kws={'label': 'F1 Score'})
plt.xlabel('Xception Weight', fontsize=12)
plt.ylabel('F3Net Weight', fontsize=12)
plt.title('Ensemble Performance Heatmap\n(Effort-CLIP weight = 1 - Xception - F3Net)', 
          fontsize=14, fontweight='bold')

# Mark best weights
best_x = w1_vals.index(round(best_weights[0], 2))
best_y = w2_vals.index(round(best_weights[1], 2))
plt.scatter([best_x], [best_y], color='red', s=200, marker='*', 
            edgecolors='white', linewidths=2, label='Best Weights')
plt.legend()

plt.tight_layout()
plt.savefig('weight_optimization_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# 3. เปรียบเทียบ Individual vs Ensemble
fig, ax = plt.subplots(figsize=(10, 6))

models_comparison = ['Xception', 'F3Net', 'Effort-CLIP', 'Ensemble\n(Best)']
accuracies = [
    results['xception']['metrics']['accuracy'],
    results['f3net']['metrics']['accuracy'],
    results['effort']['metrics']['accuracy'],
    best_metrics['accuracy']
]
f1_scores = [
    results['xception']['metrics']['f1'],
    results['f3net']['metrics']['f1'],
    results['effort']['metrics']['f1'],
    best_metrics['f1']
]

x = np.arange(len(models_comparison))
width = 0.35

bars1 = ax.bar(x - width/2, accuracies, width, label='Accuracy', color='#3498db')
bars2 = ax.bar(x + width/2, f1_scores, width, label='F1 Score', color='#e74c3c')

ax.set_ylabel('Score', fontsize=12)
ax.set_title('Individual Models vs Ensemble Performance', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(models_comparison)
ax.legend()
ax.set_ylim([0, 1])
ax.grid(axis='y', alpha=0.3)

# เพิ่มค่าบน bars
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.savefig('individual_vs_ensemble.png', dpi=150, bbox_inches='tight')
plt.show()

## 💾 บันทึกผลลัพธ์

In [ ]:
import json
from datetime import datetime

# สร้างรายงาน
report = {
    'timestamp': datetime.now().isoformat(),
    'dataset_size': len(test_data),
    'individual_models': {
        'xception': results['xception']['metrics'],
        'f3net': results['f3net']['metrics'],
        'effort': results['effort']['metrics']
    },
    'best_ensemble': {
        'weights': {
            'xception': float(best_weights[0]),
            'f3net': float(best_weights[1]),
            'effort_clip': float(best_weights[2])
        },
        'metrics': best_metrics
    },
    'top_10_configurations': sorted(
        [{
            'weights': {
                'xception': float(r['weights'][0]),
                'f3net': float(r['weights'][1]),
                'effort_clip': float(r['weights'][2])
            },
            'f1_score': float(r['score'])
        } for r in all_results],
        key=lambda x: x['f1_score'],
        reverse=True
    )[:10]
}

# บันทึกเป็น JSON
with open('weight_optimization_report.json', 'w') as f:
    json.dump(report, f, indent=2)

print("✅ Report saved to: weight_optimization_report.json")
print("\n📊 Top 10 Configurations:")
for i, config in enumerate(report['top_10_configurations'][:5], 1):
    print(f"\n{i}. F1={config['f1_score']:.4f}")
    print(f"   Xception: {config['weights']['xception']:.3f}")
    print(f"   F3Net:    {config['weights']['f3net']:.3f}")
    print(f"   Effort:   {config['weights']['effort_clip']:.3f}")

## 📝 สร้าง config.json ใหม่

In [ ]:
# สร้าง config ที่พร้อมใช้งาน
new_config = {
  "models": {
    "xception": {
      "name": "xception",
      "path": "app/models/weights/xception_best.pth",
      "description": "Fast and reliable baseline",
      "weight": round(best_weights[0], 2),
      "enabled": True
    },
    "efficientnet_b4": {
      "name": "tf_efficientnet_b4",
      "path": "app/models/weights/effnb4_best.pth",
      "description": "Balanced performance (DISABLED: incompatible checkpoint format)",
      "weight": 0.0,
      "enabled": False
    },
    "f3net": {
      "name": "f3net",
      "path": "app/models/weights/f3net_best.pth",
      "description": "Frequency-aware network with spatial attention",
      "weight": round(best_weights[1], 2),
      "enabled": True
    },
    "effort": {
      "name": "effort_clip",
      "path": "app/models/weights/effort_clip_L14_trainOn_FaceForensic.pth",
      "description": "CLIP-based multimodal detection",
      "weight": round(best_weights[2], 2),
      "enabled": True
    }
  },
  "ensemble": {
    "method": "weighted_average",
    "threshold": 0.5,
    "min_models": 2
  },
  "device": "cuda",
  "face_detection": {
    "min_confidence": 0.85,
    "min_face_size": 40
  },
  "inference": {
    "batch_size": 1,
    "generate_gradcam": False
  }
}

# บันทึก config ใหม่
with open('config_optimized.json', 'w') as f:
    json.dump(new_config, f, indent=2)

print("✅ Optimized config saved to: config_optimized.json")
print("\n📋 ใช้งานโดยคัดลอกไปแทนที่: backend/app/config.json")

## 🎯 สรุปและคำแนะนำ

### ✅ สิ่งที่ได้:
1. **ค่า weight ที่เหมาะสม** จากการทดสอบ
2. **กราฟเปรียบเทียบ** ประสิทธิภาพแต่ละโมเดล
3. **config.json ใหม่** พร้อมใช้งาน
4. **รายงานละเอียด** ใน JSON format

### 📊 การนำไปใช้:
1. ดาวน์โหลดไฟล์ `config_optimized.json`
2. คัดลอกไปแทนที่ `backend/app/config.json`
3. Restart backend server
4. ทดสอบระบบกับข้อมูลจริง

### ⚠️ หมายเหตุ:
- ผลลัพธ์ขึ้นอยู่กับ test dataset
- แนะนำให้ทดสอบกับข้อมูลหลากหลาย
- อาจต้องปรับแต่งเพิ่มตามกรณีใช้งานจริง

### 💡 Tips:
- ถ้า F1 score ใกล้เคียงกัน ให้เลือก weights ที่เรียบง่าย (เช่น 0.35, 0.30, 0.35)
- ควรทดสอบกับ cross-validation
- เก็บ log ของผลการทดสอบเพื่อเปรียบเทียบในอนาคต